# 所需环境

transformers版本号为2.1.1，pytorch为1.2.0。

In [29]:
import transformers
print(transformers.__version__)

2.1.1


在requirments.txt中，列出了所需要的其他的包，如下所示。

In [2]:
!cat requirements.txt

transformers==2.1.1
torch
numpy
tqdm
sklearn
keras
tb-nightly
future
thulac


# 目录结构以及重要文件

当前文件夹中包含的文件如下。其中**doupo**文件夹下包含“斗破苍穹”的示例任务，用以简单说明如何构建字典、tokenization、以及在一个小文件上从头训练一个指定层数的GPT模型；**pretrain**文件夹下包含预训练任务，主要用以说明如何在260万篇新闻文档上预训练一个12层的GPT2；**dataaugmentation**文件夹下包含数据增强任务，主要用以说明如何利用预训练的GPT2模型在较小的文档集上进行微调，并用于数据增强。

In [3]:
%%bash
tree -L 1 .

.
├── cache
├── config
├── dataaugmentation
├── doupo
├── eval.py
├── generate.py
├── generate_texts.py
├── LICENSE
├── pretrain
├── README.md
├── requirements.txt
├── sample
├── tokenizations
├── Train_GPT2_from_scratch.ipynb
├── train.json
├── train_on_small_file.py
└── train_single.py

7 directories, 10 files


需要重点加以说明的是，上述列表中的tokenizations文件夹，train_single.py, 和train_on_small_file.py。

## Tokenization

`tokenizations`提供了各种tokenization的方法，具体到本代码库中，使用的是该文件夹中的`tokenization_chars.py`文件。

tokenization的第一阶段是将纯文本文件依照字典文件vocab.txt进行划分为token，第二阶段是将token转化为字典文件中token对应的数字。举例来说，"[CLS]《斗破苍穹》天蚕土豆[SEP][CLS] ..."在第一阶段被分割为列表：['[CLS]', '《', '斗', '破', '苍', '穹', '》', '天', '蚕', '土', '豆', '[SEP]', '[CLS]', ...]，其中[CLS], [SEP]在vocab.txt对应一个单独的占位符；第二阶段在查找vocab.txt之后，将字符串列表转化为整型数列表：101 517 3159 4788 5721 4957 518 1921 6014 1759 6486 102 101。

如下是tokenization第一阶段的代码，输入为text字符串，输出为字符串列表result，对于形如[.\*]的特殊字符使用栈来处理：如果该特殊字符出现在vocab.txt中，则被提取为一个单独的字符串存入列表中，如161行所示；反之若没有出现在vocab.txt中，则逐字符加入到列表中，如163-165行所示。

In [7]:
!perl -ne 'print "$. $_" if ($.>=140 and $.<=176)' tokenizations/tokenization_chars.py

140     def _tokenize(self, text):
141         #pdb.set_trace()
142         stack = []
143 
144         result = []
145         i = 0
146         len_txt = len(text)
147         while(i<len_txt):
148             char = text[i]
149             if(len(stack)==0):
150                 if(char != '['):
151                     result.append(char)
152                 else:
153                     stack.append('[')
154             else:
155                 if(char == ']'):
156                     stack.append(char) # don't forget to append it firstly
157                     # process the content in the stack
158                     seq = "".join(stack)
159                     if(seq in self.vocab):
160                         # the [.*] stuff appeared in the vocabulary, e.g. [UNK], [CLS], [SEP], ...
161                         result.append(seq)
162                     else:
163                         # other [.*] stuff which are not valid element in the vocabulary
164                        

tokenization的第二部分如下所示，对第一阶段获得的result列表中的token字符串进行查表操作，如果没有出现在vocab.txt中，那么就以[UNK]对应的id来代替。

In [18]:
!perl -ne 'print "$. $_" if ($.>=178 and $.<=182)' tokenizations/tokenization_chars.py

178     def _convert_token_to_id(self, token):
179         """ Converts a token (str/unicode) in an id using the vocab. """
180         #if(token not in self.vocab):
181         #    print(token)
182         return self.vocab.get(token, self.vocab.get(self.unk_token))


注意，本代码库中的tokenization方法主要为中文设计，如果训练数据中混杂有英文字符，那么就将英文单词按照char level进行划分，例如"word"会被"w","o","r","d"四个字符来代替。不同于BPE的方法，我们认为这样处理是最节省计算资源的，比起BPE能够更从容应对几十GB的中文训练语料。从具体的实际效果来看，在斗破苍穹的语料上进行tokenization，普通方法需要耗时77.9秒，使用了我们的方法之后，耗时12.1秒，用时为原来的15%。

## 两种训练方法

`train_single.py`：当输入的训练数据为一篇完整的长文档（例如小说）或是一个单一的大型文档，此时使用`train_single.py`。有两方面的原因。（1）train_single.py在训练过程中不对training samples进行random shuffle，保持training samples之间的顺序，因此完整的长文档使用train_single.py。（2）对顺序无关的超多的样本，例如包含260万篇新闻的大型训练集，在训练过程中进行random shuffle的代价是巨大的，包括shuffle的代价，重新划分training sample的代价和tokenization的代价，并且在大型训练集上我们进行预训练的轮数有限，1到2轮就能获得一个较好的预训练模型。综合考虑shuffle的代价和收益，因此也将大型训练集中的多个样本拼接为一个单一的大型文档，使用train_single.py。我们在《从头训练一个GPT2模型》一节对此进行更详细的说明。

`train_on_small_file.py`：当输入的训练数据为多个样本，并且这多个样本的总体积较小时，将这些样本汇总到一个文件，每个样本占据一行，然后使用`train_on_small_file.py`。有如下两方面原因：（1）train_on_small_file.py针对每个样本建立一个单独的training sample送入Transformer Encoder中，如果一个样本的长度不足Transformer Encoder的长度（在本代码库中用n_ctx表示），那么不足部分以\[PAD\]来补足；超出n_ctx的部分直接截断丢弃不用。这一点和train_single.py不同，train_single.py中多个较短的样本可能会占据一个Transformer Encoder的输入长度（n_ctx）。（2）train_on_small_file.py会在不同的训练轮次中，对training samples进行random shuffle，以提高训练质量。由于train_on_small_file.py的训练数据体积较小，样本数也较小，因此每轮进行random shuffle是完全可行的。所以，如果是唐诗、宋词、现代诗、意图增强等文本数据，由于其规模较小，完全可以使用`train_on_small_file.py`来完成训练或者微调，这样得到的模型质量要高于`train_single.py`训练得到的模型。我们在《微调GPT2模型进行数据增强》一节对此进行更详细的说明。

# 从头训练一个GPT2模型

In [4]:
!perl -ne 'print "$. $_" if ($.>=18 and $.<=67)' train_single.py

18 def build_files(raw_data_path, divide_path, tokenized_data_path, full_tokenizer, num_pieces):
19     if not os.path.exists(tokenized_data_path):
20         os.mkdir(tokenized_data_path)
21     if not os.path.exists(divide_path):
22         os.mkdir(divide_path)
23     print("now time: ", datetime.now())
24     print("begin to divide raw text ...")
25 
26     total_line_num = 0
27     with open(raw_data_path, 'r', encoding='utf8') as f:
28         for line in f:
29             total_line_num+=1
30 
31     writers = [open(divide_path + 'divide_piece_{}.txt'.format(i), 'w') for i in range(0,num_pieces)]
32 
33     with open(raw_data_path, 'r', encoding='utf8') as f:
34         line_num = 0
35         for line in f:
36             writers[line_num % num_pieces].write("%s" % line)
37             line_num += 1
38     
39     for i in range(0, num_pieces):
40         writers[i].close()
41     
42     print('now time: ', datetime.now())
43     print("begin making tokenization ...")
44     f

In [9]:
!ls

config		   model		 tokenized
divide		   outputs		 Train_GPT2_from_scratch.ipynb
format_raw_txt.py  outputs_from_scratch  train.sh
generate.sh	   rawdata		 vocab.txt


In [13]:
!bash train.sh

setting config/vocab.txt and config/model_config.json
I0414 14:48:02.821990 140337405437760 file_utils.py:39] PyTorch version 1.2.0 available.
args:
Namespace(batch_size=32, device='0,1', divide_path='doupo/divide/', epochs=30, fp16=False, fp16_opt_level='O1', gradient_accumulation=1, ignore_intermediate_epoch_model=True, log_step=20, lr=0.00015, max_grad_norm=1.0, model_config='doupo/config/model_config.json', num_pieces=1, output_dir='doupo/model/', pretrained_model='', raw=False, raw_data_path='doupo/rawdata/train.txt', segment=False, stride=512, tokenized_data_path='doupo/tokenized/', tokenizer_path='doupo/config/vocab.txt', warmup_steps=2000)
config:
{
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "finetuning_task": null,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "n_ctx": 512,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 10,
  "n_positions": 512,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_he

In [17]:
!cd doupo;bash generate.sh;cd ..

I0415 00:18:07.675502 140303028275008 file_utils.py:39] PyTorch version 1.2.0 available.
args:
Namespace(batch_size=1, device='0', fast_pattern=False, length=100, model_config='doupo/model/final_model/model_config.json', model_path='doupo/model/final_model/', no_wordpiece=False, nsamples=10, prefix='萧炎', repetition_penalty=1.0, save_samples=True, save_samples_path='doupo/outputs/', segment=False, temperature=0.8, tokenizer_path='doupo/config/vocab.txt', topk=50, topp=0)
I0415 00:18:07.959635 140303028275008 configuration_utils.py:148] loading configuration file doupo/model/final_model/config.json
I0415 00:18:07.960070 140303028275008 configuration_utils.py:168] Model config {
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "finetuning_task": null,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "n_ctx": 512,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 10,
  "n_positions": 512,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past":

In [25]:
!ls

config	format_raw_txt.py  model    outputs_from_scratch  tokenized  vocab.txt
divide	generate.sh	   outputs  rawdata		  train.sh


# 微调GPT2模型进行数据增强

In [ ]:
!ls